# The Battle of Neighborhoods

## Introduction

The problem we want to address is to identify the location that is suitable to open a new resturant in the City of Toronto.


## Data

The data we shall use is the neighborhoods of Toronto (from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M); the location information for each neighborhoods (from `geocoder`) and the venues information for each neighborhoods (from *FourSquare*).

We shall fit the Multi-dimensional Linear Regression where the features are the number of venues which is not a restaurant; and the dependent variable is the number of venues which is a restaurant. Then we use this model to predict the expected number of restaurants for each neighbourhood. If the actual value is lower than the expected value, then we shall consider it is a good location to open a new restaurant as there are still markets in that neighbourhood.  In contrast, if their actual value is larger than expected value, then we shall consider it is a bad location as the competition is too high in that neighbourhood. Then we can sort the result to see which location is good to open a new restaurant.